In [3]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score


## Get the Data and Labels

In [4]:
num_testsub = 32
num_classes = 4
num_channel = 40
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = 'data/data_prepared/data_norm_bhat/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [5]:
data_dict = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [6]:
valence = np.load('data/data_prepared/labels/label_class_0.npy')
arousal = np.load('data/data_prepared/labels/label_class_1.npy')
dominance = np.load('data/data_prepared/labels/label_class_2.npy')
liking = np.load('data/data_prepared/labels/label_class_3.npy')

In [7]:
valence.shape

(1280,)

In [8]:
data_dict['sub01'].shape

(40, 40, 99)

## Create train test split

### Data

In [27]:
# training with 22 participants each has 40 videos = 880
train = np.zeros((0,40,99))
train[:] = np.nan
#train1 = 
test = np.zeros((0,40,99))
test[:] = np.nan

participation_counter = 0
for sub in data_dict.keys():
    if participation_counter < 22:
        train = np.concatenate((train, data_dict[sub]), axis=0)
    else:
        test = np.concatenate((test, data_dict[sub]), axis=0)
    participation_counter = participation_counter + 1

In [29]:
np.isnan(train).any()

False

In [30]:
np.isnan(test).any()

False

In [31]:
train.shape

(880, 40, 99)

In [32]:
test.shape

(400, 40, 99)

### Labels

In [40]:
y_train_va = valence[: 880]
y_train_ar = arousal[: 880]
y_train_do = dominance[: 880]
y_train_li = liking[: 880]

y_test_va = valence[880: ]
y_test_ar = arousal[880: ]
y_test_do = dominance[880: ]
y_test_li = liking[880: ]

In [41]:
print(y_train_va.shape)
print(y_train_ar.shape)
print(y_train_do.shape)
print(y_train_li.shape)
print(y_test_va.shape)
print(y_test_ar.shape)
print(y_test_do.shape)
print(y_test_li.shape)

(880,)
(880,)
(880,)
(880,)
(400,)
(400,)
(400,)
(400,)


In [134]:
train.reshape(-1, 40*99).shape

(880, 3960)

## KNN

In [50]:
# Define KNN based model for augmented data
def knnmodel(xtrain,ytrain,xtest,ytest,kval):
  
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtest.shape,ytest.shape)

  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval)
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  #print(classification_report(ytest, y_pred))
  #print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [ ]:
trainlabel_list = [y_train_va, y_train_ar, y_train_do, y_train_li]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
score_k_value = tuple() 



for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel(train.reshape(-1, 40*99),traain,test.reshape(-1, 40*99),tesst,kval = k)
    scorelist.append((sc,k))
    score_k_value = score_k_value + (sc, k)
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4

#for i,name in zip(scorelist,names):
#    print ("Accuracy of ",name ,i)

In [132]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(53.75, 19)
(56.75, 20)
(58.75, 7)
(63.0, 49)


In [129]:
scorelist = [(15,0),(8,0),(88,0),(40,9),
             (5,6),(67,9),(7,6),(23,0),
             (12,2),(3,1),(5,3),(78,8)]
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4